In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/covid_19_data.csv', parse_dates=['ObservationDate', 'Last Update'])

In [ ]:
df

In [ ]:
df.dtypes

# padronizando nome das colunas

In [ ]:
import re 

def corrige_colunas(col_name):
  return re.sub(r'[/| ]', '', col_name).lower()

In [ ]:
corrige_colunas('aaBAD/P ou') # teste

In [ ]:
df.columns = [corrige_colunas(col) for col in df.columns]

In [ ]:
df

# selecionando apenas o brasil

In [ ]:
df.loc[df.countryregion == 'Brazil']

In [ ]:
brasil = df.loc[(df.countryregion == 'Brazil') & (df.confirmed > 0)]

In [ ]:
brasil

# casos confirmados

In [ ]:
px.line(brasil, 'observationdate', 'confirmed', title='Casos Confirmados no Brasil')

# novos casos por dia

In [ ]:
brasil['novoscasos'] = list(map(lambda x: 0 if (x == 0) else brasil['confirmed'].iloc[x] - brasil['confirmed'].iloc[x - 1], np.arange(brasil.shape[0])))

In [ ]:
px.line(brasil, 'observationdate', 'novoscasos', title='Novos Casos Por Dia')

# mortes

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = brasil.observationdate, y = brasil.deaths, name = 'Mortes', mode = 'lines+markers', line = {'color''': 'red'}))
fig.update_layout(title = 'Mortes por COVID-19 no Brasil')
fig.show()

# taxa de crescimento

In [ ]:
def taxa_crescimento(data, variable, data_inicio=None, data_fim=None):
  if data_inicio == None:
    data_inicio = data.observationdate.loc[data[variable] > 0].min()
  else:
    data_inicio = pd.to_datetime(data_inicio)

  if data_fim == None:
    data_fim = data.observationdate.iloc[-1]
  else:
    data_fim = pd.to_datetime(data_fim)

  passado = data.loc[data.observationdate == data_inicio, variable].values[0]
  presente = data.loc[data.observationdate == data_fim, variable].values[0]

  n = (data_fim - data_inicio).days

  taxa = (presente/passado) ** (1/n) - 1

  return taxa*100

In [ ]:
taxa_crescimento(brasil, 'confirmed') # taxa media em todo o periodo

In [ ]:
def taxa_crescimento_diario(data, variable, data_inicio=None):
  if data_inicio == None:
    data_inicio = data.observationdate.loc[data[variable] > 0].min()
  else:
    data_inicio = pd.to_datetime(data_inicio)

  data_fim = data.observationdate.max()

  n = (data_fim - data_inicio).days

  taxas = list(map(lambda x: (data[variable].iloc[x] - data[variable].iloc[x - 1]) / data[variable].iloc[x - 1], range(1, n+1)))

  return np.array(taxas) * 100

In [ ]:
taxa_dia = taxa_crescimento_diario(brasil, 'confirmed')

In [ ]:
taxa_dia

In [ ]:
primeiro_dia = brasil.observationdate.loc[brasil.confirmed > 0].min()

px.line(x = pd.date_range(primeiro_dia, brasil.observationdate.max())[1:], y = taxa_dia, title = 'Taxa De Casos Confirmados No Brasil')

# predicoes

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt

In [ ]:
confirmados = brasil.confirmed
confirmados.index = brasil.observationdate
confirmados

In [ ]:
res = seasonal_decompose(confirmados)

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10, 8))

ax1.plot(res.observed)
ax2.plot(res.trend)
ax3.plot(res.seasonal)
ax4.plot(confirmados.index, res.resid)
ax4.axhline(0, linestyle='dashed', c='black')
plt.show()

# modelo arima

In [ ]:
from pmdarima.arima import auto_arima
modelo = auto_arima(confirmados)

In [ ]:
fig = go.Figure(go.Scatter(x = confirmados.index, y = confirmados, name = 'Observados'))
fig.add_trace(go.Scatter(x = confirmados.index, y = modelo.predict_in_sample(), name = 'Preditos'))
fig.add_trace(go.Scatter(x = pd.date_range('2020-05-20', '2020-06-20'), y = modelo.predict(31), name = 'Forecast'))
fig.update_layout(title = 'Previsao de Casos Confirmados Para os Proximos 30 dias')
fig.show()

# modelo crescimento

In [ ]:
from fbprophet import Prophet

In [ ]:
train = confirmados.reset_index()[:-5]
test = confirmados.reset_index()[-5:]

train.rename(columns = {'observationdate': 'ds', 'confirmed': 'y'}, implace = True)
test.rename(columns = {'observationdate': 'ds', 'confirmed': 'y'}, implace = True)

profeta = Prophet(growth = 'logistic', changepoints = ['2020-03-21', '2020-03-30', '2020-04-25', '2020-05-03', '2020-05-10'])

pop = 211463256
train['cap'] = pop

profeta.fit(train)

future_dates = profeta.make_future_dataframe(periods=200)
future_dates['cap'] = pop
forecast = profeta.predict(future_dates)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = forecast.ds, y = forecast.yhat, name = 'Predicao'))
fig.add_trace(go.Scatter(x = test.index, y = test, name = 'Observados - teste'))
fig.add_trace(go.Scatter(x = train.ds, y = train.y, name = 'Observados - treino'))
fig.update_layout(title = 'Predicoes De Casos Confirmados No Brasil')
fig.show()